<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Data Preprocessing in Python: Data Integration and Reshaping

By now, we  know how to collect data from different places, and clean/format them as needed. While the cleaning and formatting is done for each data source, we finally need to integrate all the sources into one to start the real analytical work. Let's get the data on [democracy index](https://en.wikipedia.org/wiki/Democracy_Index) by _The Economist_:

In [1]:
# importing pandas for the notebook
import pandas as pd

#link for the tables
demoLink="https://en.wikipedia.org/wiki/Democracy_Index"

# fetching the tables
demodata=pd.read_html(demoLink,header=0,flavor="bs4",attrs={'class':"wikitable sortable"})

You should remember by now that **demodata** is a _list_ of all the sortable tables from that URL. Let me recover the one we want:

In [2]:
demoVars=demodata[4].copy()

In this last session on Python, we want to prepare data on the world countries and their level of democracy. I will add some variables from other tables to this one later. Take a look at what you have now:

In [3]:
# first rows
demoVars.head()

,Rank,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Δ Rank,Country,Regime type,Overall score,Δ Score,Elec­toral pro­cess and plura­lism,Func­tioning of govern­ment,Poli­tical partici­pation,Poli­tical cul­ture,Civil liber­ties
0,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies,Full democracies
1,1,NaN,Norway,Full democracy,9.81,0.06,10.00,9.64,10.00,10.00,9.41
2,2,NaN,Iceland,Full democracy,9.37,0.21,10.00,8.57,8.89,10.00,9.41
3,3,NaN,Sweden,Full democracy,9.26,0.13,9.58,9.29,8.33,10.00,9.12
4,4,NaN,New Zealand,Full democracy,9.25,0.01,10.00,8.93,8.89,8.75,9.71


Let's start by getting rid of the columns we will not use:

In [4]:
#column positions to drop
whichToDrop=[0,1,5]

#dropping and updating the data frame
demoVars.drop(labels=demoVars.columns[whichToDrop],axis=1,inplace=True)

Let's take a look at the columns names:

In [5]:
# these are:
demoVars.columns

Index(['Country', 'Regime type', 'Overall score',
       'Elec­toral pro­cess and plura­lism', 'Func­tioning of govern­ment',
       'Poli­tical partici­pation', 'Poli­tical cul­ture', 'Civil liber­ties'],
      dtype='object')

The previous result shows nothing relevant, but the pandas data frame shows _unneeded dashes_ in the column names. Let's **check the presence of special characters**:

In [6]:
# this is a column name with dashes:
import re
re.sub("\s","",demoVars.columns[5])

'Poli\xadticalpartici\xadpation'

The result shows you some **hidden characters**. Let's use that info with the pandas replace:

In [7]:
# using pandas own replace function
demoVars.columns=demoVars.columns.str.replace("\s|\xad","",regex=True)

# current colum names
demoVars.columns

Index(['Country', 'Regimetype', 'Overallscore', 'Electoralprocessandpluralism',
       'Functioningofgovernment', 'Politicalparticipation', 'Politicalculture',
       'Civilliberties'],
      dtype='object')

Let's clean the data contents. Notice that in the website some labels that are not needed are present in this case. Let's check the frequency table of "Regime Type" to try to identify the wrong labels that are affecting the data frame to delete those *rows*: 

In [8]:
# frequency table using "value_counts"
demoVars['Regimetype'].value_counts()

Authoritarian            57
Flawed democracy         52
Hybrid regime            35
Full democracy           23
Full democracies          1
Flawed democracies        1
Hybrid regimes            1
Authoritarian regimes     1
Name: Regimetype, dtype: int64

In [9]:
# these are the wrong ones:
demoVars['Regimetype'].value_counts().index[4:]

Index(['Full democracies', 'Flawed democracies', 'Hybrid regimes',
       'Authoritarian regimes'],
      dtype='object')

In [10]:
#saving the wrong ones:
byeValues=demoVars['Regimetype'].value_counts().index[4:]

Now that we know which ones are not needed, we can **filter** the data frame **rows** using pandas' **isnin**:

In [11]:
# use "~" to request the "opposite" rows
demoVars = demoVars[~demoVars['Regimetype'].isin(byeValues)]

# data frame after
demoVars

,Country,Regimetype,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties
1,Norway,Full democracy,9.81,10.00,9.64,10.00,10.00,9.41
2,Iceland,Full democracy,9.37,10.00,8.57,8.89,10.00,9.41
3,Sweden,Full democracy,9.26,9.58,9.29,8.33,10.00,9.12
4,New Zealand,Full democracy,9.25,10.00,8.93,8.89,8.75,9.71
5,Canada,Full democracy,9.24,9.58,8.93,8.89,9.38,9.41
...,...,...,...,...,...,...,...,...
166,Chad,Authoritarian,1.55,0.00,0.00,1.67,3.75,2.35
167,Syria,Authoritarian,1.43,0.00,0.00,2.78,4.38,0.00
168,Central African Republic,Authoritarian,1.32,1.25,0.00,1.11,1.88,2.35
169,Democratic Republic of the Congo,Authoritarian,1.13,0.00,0.00,1.67,3.13,0.88


Notice that the index **0** has dissapeared, and that even though you have 167 countries now, the last one has index **170**. When you filter rows that will happen; so it is better to **reset the indexes of the data frame**:

In [12]:
# new indexes will appear
demoVars.reset_index(drop=True,inplace=True)

In [13]:
demoVars

,Country,Regimetype,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties
0,Norway,Full democracy,9.81,10.00,9.64,10.00,10.00,9.41
1,Iceland,Full democracy,9.37,10.00,8.57,8.89,10.00,9.41
2,Sweden,Full democracy,9.26,9.58,9.29,8.33,10.00,9.12
3,New Zealand,Full democracy,9.25,10.00,8.93,8.89,8.75,9.71
4,Canada,Full democracy,9.24,9.58,8.93,8.89,9.38,9.41
...,...,...,...,...,...,...,...,...
162,Chad,Authoritarian,1.55,0.00,0.00,1.67,3.75,2.35
163,Syria,Authoritarian,1.43,0.00,0.00,2.78,4.38,0.00
164,Central African Republic,Authoritarian,1.32,1.25,0.00,1.11,1.88,2.35
165,Democratic Republic of the Congo,Authoritarian,1.13,0.00,0.00,1.67,3.13,0.88


Let's save the file as in CSV format:

In [ ]:
demoVars.to_csv("demoVars.csv",index=False)

## Exercise 1:

- Go to this link: [https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes).
- Scrap the table of countries, and keep country names, **ISO codes** and **Internet codes**.
- Prepare a CSV file with the clean and formatted data, and upload that file into GitHub.
- Read the file from Github and add the ISO codes and internet codes to the table of democracy index.

# Integrating Data Frames

Integrating data sets needs the following considerations:

* Merging is done on two data frames.
* You need a column in each data frame with that share the same exact and unique values. The column names or titles need not be the same.
* The merged table shows by default the mutual coincidences; but you can also request the values not matched, which will help you detect possible extra cleaning.
* Pandas differentiates the **left** from the **right** data frames.

Let me bring the new data frame with ISO codes and internet codes:

In [14]:
# link to the data in CSV format
linkDataIso='https://github.com/EvansDataScience/CTforGA_integrating/raw/main/isodata.csv'

# using 'read_csv' with a link
DataIso=pd.read_csv(linkDataIso)

In [15]:
#Check the current names:
DataIso.columns

Index(['Countryname', 'Officialstatename', 'InternetccTLD', 'iso2', 'iso3'], dtype='object')

* **Option one**(default): merge only the coincidences:

In [16]:
# notice the key column are country names, but the column names are different:
allData=demoVars.merge(DataIso,left_on="Country",right_on="Countryname") 

Once the merged has taken place, check the differences:

In [17]:
# number of rows (countries) before merge
demoVars.shape

(167, 8)

In [18]:
# number of rows after merge
allData.shape

(138, 13)

The previous merge has **lost several rows** (countries).

* **Option two**: you want to keep all the rows, matched and unmatched, from **both** data frames:

In [19]:
# merge 'outer' keeps all matched and unmatched
# merge 'indicator' will tell you if the key was found in 'both' or only in one of them
allData=demoVars.merge(DataIso,left_on="Country",right_on="Countryname",how='outer',indicator='True')

The _allData_ dataframe has now a  column named 'True', the **indicator**:

In [20]:
allData.columns

Index(['Country', 'Regimetype', 'Overallscore', 'Electoralprocessandpluralism',
       'Functioningofgovernment', 'Politicalparticipation', 'Politicalculture',
       'Civilliberties', 'Countryname', 'Officialstatename', 'InternetccTLD',
       'iso2', 'iso3', 'True'],
      dtype='object')

In [21]:
# explore matched and unmatched counts:
allData['True'].value_counts()

both          138
right_only    111
left_only      29
Name: True, dtype: int64

Our main task is to add columns to the Democracy Index data. So these are the countries from The Economist that did not find a match:

In [22]:
allData[allData['True']=='left_only'].Country

9                           Netherlands
10                               Taiwan
15                       United Kingdom
22                          South Korea
24                        United States
30                       Czech Republic
31                           Cape Verde
54                          Philippines
62                   Dominican Republic
79                              Moldova
92                             Tanzania
93                              Bolivia
102                              Gambia
108                         Ivory Coast
112                           Palestine
123                              Russia
124                               Niger
128               Republic of the Congo
131                             Comoros
136                             Vietnam
142                           Venezuela
144                United Arab Emirates
148                               Sudan
151                                Iran
160                                Laos


Those names should be here, but written differently:

In [23]:
allData[allData['True']=='right_only'].Countryname

167               Åland Islands
168              American Samoa
169                     Andorra
170                    Anguilla
171                  Antarctica
                 ...           
273                    Viet Nam
274    Virgin Islands (British)
275       Virgin Islands (U.S.)
276           Wallis and Futuna
277              Western Sahara
Name: Countryname, Length: 111, dtype: object

The next steps might seem a little laborious, combining computational thinking adn finally some brute force. The first tool we need is another library allows fuzzy merging: [TheFuzz](https://github.com/seatgeek/thefuzz).

The library has two functions of interest:_extractOne_ and _extract_. Let me show you how they work:

In [ ]:
#!pip install thefuzz

In [24]:
# The countries unmatched
UnmatchedLeft=allData[allData['True']=='left_only'].Country.to_list()
UnmatchedRight=allData[allData['True']=='right_only'].Countryname.to_list()

This is a country name fro the economist that did not get a match in the ISO data:

In [25]:
UnmatchedLeft[0]

'Netherlands'

Let's find the best match from the ISO data:

In [26]:
from thefuzz import process

process.extractOne(UnmatchedLeft[0], UnmatchedRight)

('Netherlands (the)', 95)

Let's find the three best matches from the ISO data:

In [27]:
process.extract(UnmatchedLeft[0], UnmatchedRight,limit=3)

[('Netherlands (the)', 95),
 ('Greenland', 60),
 ('Saint Helena\xa0Ascension Island\xa0Tristan da Cunha', 58)]

These operations will not be done blindly. We will need to see what matches are done:

In [28]:
UnmatchedLeft=allData[allData['True']=='left_only'].Country.to_list()
UnmatchedLeft

['Netherlands',
 'Taiwan',
 'United Kingdom',
 'South Korea',
 'United States',
 'Czech Republic',
 'Cape Verde',
 'Philippines',
 'Dominican Republic',
 'Moldova',
 'Tanzania',
 'Bolivia',
 'Gambia',
 'Ivory Coast',
 'Palestine',
 'Russia',
 'Niger',
 'Republic of the Congo',
 'Comoros',
 'Vietnam',
 'Venezuela',
 'United Arab Emirates',
 'Sudan',
 'Iran',
 'Laos',
 'Syria',
 'Central African Republic',
 'Democratic Republic of the Congo',
 'North Korea']

In [29]:
[(left, process.extractOne(left, UnmatchedRight)) for left in sorted(UnmatchedLeft)]

[('Bolivia', ('Bolivia (Plurinational State of)', 90)),
 ('Cape Verde', ('Cabo Verde', 80)),
 ('Central African Republic', ('Central African Republic (the)', 95)),
 ('Comoros', ('Comoros (the)', 90)),
 ('Czech Republic', ('Central African Republic (the)', 86)),
 ('Democratic Republic of the Congo',
  ('Congo (the Democratic Republic of the)', 95)),
 ('Dominican Republic', ('Dominican Republic (the)', 95)),
 ('Gambia', ('Gambia (the)', 90)),
 ('Iran', ('Iran (Islamic Republic of)', 90)),
 ('Ivory Coast', ("Côte d'Ivoire", 58)),
 ('Laos', ('Barbados', 68)),
 ('Moldova', ('Moldova (the Republic of)', 90)),
 ('Netherlands', ('Netherlands (the)', 95)),
 ('Niger', ('Niger (the)', 90)),
 ('North Korea', ("Korea (the Democratic People's Republic of)", 86)),
 ('Palestine', ('Palestine, State of', 90)),
 ('Philippines', ('Philippines (the)', 95)),
 ('Republic of the Congo', ('Bahamas (the)', 86)),
 ('Russia', ('Russian Federation (the)', 90)),
 ('South Korea', ("Korea (the Democratic People's Re

These are the totally wrong matches:

In [31]:
# this is a list of tuples:
TotallyWrong=[('Czech Republic', ('Central African Republic (the)', 86)),
              ('Laos', ('Barbados', 68)),
              ('Republic of the Congo', ('Bahamas (the)', 86)),
              ('South Korea', ("Korea (the Democratic People's Republic of)", 86)),
              ('Sudan', ('South Sudan', 90)),
              ('United States', ('United States Minor Outlying Islands (the)', 90))]

Let me make a list with all the names in the democracy index countries that must be omitted currently:

In [32]:
omitLeft=[leftName for (leftName,rightFuzzy) in TotallyWrong] #parenthesis not needed
omitLeft

['Czech Republic',
 'Laos',
 'Republic of the Congo',
 'South Korea',
 'Sudan',
 'United States']

Create a dictionary with the changes that will be done:

In [33]:
# process.extractOne(left, UnmatchedRight)[0] IS JUST THE COUNTRY NAME TO THE RIGHT
# [0] just took first element in tuple.

# left for left in UnmatchedLeft IS JUST EVERY COUNTRY NAME TO THE LEFT

# if left not in omitLeft IS A CONDITION

{process.extractOne(left, UnmatchedRight)[0]:left for left in UnmatchedLeft if left not in omitLeft}

{'Netherlands (the)': 'Netherlands',
 'Taiwan (Province of China)': 'Taiwan',
 'United Kingdom of Great Britain and Northern Ireland (the)': 'United Kingdom',
 'Cabo Verde': 'Cape Verde',
 'Philippines (the)': 'Philippines',
 'Dominican Republic (the)': 'Dominican Republic',
 'Moldova (the Republic of)': 'Moldova',
 'Tanzania, the United Republic of': 'Tanzania',
 'Bolivia (Plurinational State of)': 'Bolivia',
 'Gambia (the)': 'Gambia',
 "Côte d'Ivoire": 'Ivory Coast',
 'Palestine, State of': 'Palestine',
 'Russian Federation (the)': 'Russia',
 'Niger (the)': 'Niger',
 'Comoros (the)': 'Comoros',
 'Viet Nam': 'Vietnam',
 'Venezuela (Bolivarian Republic of)': 'Venezuela',
 'United Arab Emirates (the)': 'United Arab Emirates',
 'Iran (Islamic Republic of)': 'Iran',
 'Syrian Arab Republic (the)': 'Syria',
 'Central African Republic (the)': 'Central African Republic',
 'Congo (the Democratic Republic of the)': 'Democratic Republic of the Congo',
 "Korea (the Democratic People's Republic of

Let's save the dict above and use it to replace:

In [34]:
changesRight={process.extractOne(left, UnmatchedRight)[0]:left for left in UnmatchedLeft if left not in omitLeft}
DataIso.Countryname.replace(changesRight,inplace=True)

DataIso has new country names. Let's prepare manually the remaining changes:

In [35]:
# dict of manual changes
bruteForceChanges={'Korea (the Republic of)':'South Korea', 
                'United States of America (the)':'United States',
                'Czechia':'Czech Republic', 
                'Congo (the)':'Republic of the Congo',
                'Sudan (the)':'Sudan',
                "Lao People's Democratic Republic (the)":'Laos'}

# replacing
DataIso.Countryname.replace(bruteForceChanges,inplace=True)

In [37]:
DataIso.Countryname.to_list()

['Afghanistan',
 'Åland Islands',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas (the)',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire\xa0Sint Eustatius\xa0Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory (the)',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cape Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands (the)',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos (Keeling) Islands (the)',
 'Colombia',
 'Comoros',
 'Democratic Republic of the Congo',
 'Republic of the Congo',
 'Cook Islands (the)',
 'Costa Rica',
 'Ivory Coast',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 

Let's redo the merging with full coincidences (option 1):

In [38]:
# redoing merge
allData=demoVars.merge(DataIso,left_on="Country",right_on="Countryname")

# current dimension
allData.shape

(167, 13)

In [39]:
allData

,Country,Regimetype,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Countryname,Officialstatename,InternetccTLD,iso2,iso3
0,Norway,Full democracy,9.81,10.00,9.64,10.00,10.00,9.41,Norway,The Kingdom of Norway,.no,NO,NOR
1,Iceland,Full democracy,9.37,10.00,8.57,8.89,10.00,9.41,Iceland,Iceland,.is,IS,ISL
2,Sweden,Full democracy,9.26,9.58,9.29,8.33,10.00,9.12,Sweden,The Kingdom of Sweden,.se,SE,SWE
3,New Zealand,Full democracy,9.25,10.00,8.93,8.89,8.75,9.71,New Zealand,New Zealand,.nz,NZ,NZL
4,Canada,Full democracy,9.24,9.58,8.93,8.89,9.38,9.41,Canada,Canada,.ca,CA,CAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Chad,Authoritarian,1.55,0.00,0.00,1.67,3.75,2.35,Chad,The Republic of Chad,.td,TD,TCD
163,Syria,Authoritarian,1.43,0.00,0.00,2.78,4.38,0.00,Syria,The Syrian Arab Republic,.sy,SY,SYR
164,Central African Republic,Authoritarian,1.32,1.25,0.00,1.11,1.88,2.35,Central African Republic,The Central African Republic,.cf,CF,CAF
165,Democratic Republic of the Congo,Authoritarian,1.13,0.00,0.00,1.67,3.13,0.88,Democratic Republic of the Congo,The Democratic Republic of the Congo,.cd,CD,COD


Here, we could drop the columns that are not needed:

In [40]:
allData.drop(["Countryname"],axis=1,inplace=True)
allData

,Country,Regimetype,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Officialstatename,InternetccTLD,iso2,iso3
0,Norway,Full democracy,9.81,10.00,9.64,10.00,10.00,9.41,The Kingdom of Norway,.no,NO,NOR
1,Iceland,Full democracy,9.37,10.00,8.57,8.89,10.00,9.41,Iceland,.is,IS,ISL
2,Sweden,Full democracy,9.26,9.58,9.29,8.33,10.00,9.12,The Kingdom of Sweden,.se,SE,SWE
3,New Zealand,Full democracy,9.25,10.00,8.93,8.89,8.75,9.71,New Zealand,.nz,NZ,NZL
4,Canada,Full democracy,9.24,9.58,8.93,8.89,9.38,9.41,Canada,.ca,CA,CAN
...,...,...,...,...,...,...,...,...,...,...,...,...
162,Chad,Authoritarian,1.55,0.00,0.00,1.67,3.75,2.35,The Republic of Chad,.td,TD,TCD
163,Syria,Authoritarian,1.43,0.00,0.00,2.78,4.38,0.00,The Syrian Arab Republic,.sy,SY,SYR
164,Central African Republic,Authoritarian,1.32,1.25,0.00,1.11,1.88,2.35,The Central African Republic,.cf,CF,CAF
165,Democratic Republic of the Congo,Authoritarian,1.13,0.00,0.00,1.67,3.13,0.88,The Democratic Republic of the Congo,.cd,CD,COD


Let's save this merged data frame in CSV format:

In [ ]:
allData.to_csv('allData.csv',index=False)

## Exercise 2:

- Go to this link: [https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population).
- Scrap the table of countries, and keep country names, and region.
- Make sure the country names match exactly the ones from the table of democracy index.
- Merge this table with the allData data frame (just add the region)
- Prepare a CSV file with the clean, formatted and merged data, and upload that file into GitHub.

# RESHAPING

### Wide and Long format

Let's open **allDataFull**, whose structure has the **WIDE** format. In this format, the variables are in every column, the most traditional one for spreadsheet users. 

In [ ]:
allDataFull=pd.read_csv('https://github.com/EvansDataScience/CTforGA_integrating/raw/main/allDataFull.csv')
allDataFull

This is the final dataframe, let's check the data types, in case they need formatting:

In [ ]:
allDataFull.info()

We should turn the texts of the regime type column into ordinal levels:

In [ ]:
from pandas.api.types import CategoricalDtype

# create data type - notice ascending order of levels
levelsDemo = CategoricalDtype(categories=['Authoritarian', 'Hybrid regime', 'Flawed democracy', 'Full democracy'], 
                              ordered=True)

# make the change:
allDataFull.Regimetype=allDataFull.Regimetype.astype(levelsDemo)

# see result:
allDataFull.Regimetype

Once you have the right data types, you can request stats:

In [ ]:
allDataFull.describe(include='all')
# try? 
# allDataFull.describe()

Let's see turn it into a **LONG** format:

In [ ]:
allDataFull_LONG=allDataFull.melt(id_vars=['Country','Region',"Regimetype",'iso3','iso2','InternetccTLD'])
allDataFull_LONG

The amount of of rows multiplies, but **some** codes are easier to implement with long format (or can only be implemented in that format), like plotting:

In [ ]:
import plotnine as p9


base=p9.ggplot(data=allDataFull_LONG)
base + p9.geom_boxplot(p9.aes(x='variable',y='value')) + p9.coord_flip()

Using the original format (wide), you could only ploy one bar using the previous *grammar*:

In [ ]:
base=p9.ggplot(data=allDataFull)
base + p9.geom_boxplot(p9.aes(x=1,y='Overallscore')) + p9.coord_flip()

Nonetheless, there are packages in Python that use the wide format for plotting (i.e. _seaborn_):

In [ ]:
import seaborn as sns
  
whatVars= ['Overallscore', 'Electoralprocessandpluralism',
           'Functioningofgovernment', 'Politicalparticipation', 'Politicalculture',
           'Civilliberties']
sns.boxplot(data=allDataFull[whatVars], 
                 orient="h",color='white')

# Aggregating

Sometimes you need to aggregate your original data frame, that is, aggregate using a particular function some columns by a particular group . 

For example, if we wanted the average of the overall score of democracy by region, and get the result as a data frame:

In [ ]:
allDataFull.groupby("Region")[['Overallscore']].agg('mean').reset_index()

What if we wanted the average of the overall score of democracy by region and by regime type, and get the result as a data frame:

In [ ]:
allDataFull.groupby(["Region","Regimetype"])[['Overallscore']].agg('mean').reset_index()

Notice the if you do not use _reset_index()_, you will not get the groups as columns, but as row indexes:

In [ ]:
allDataFull.groupby(["Region","Regimetype"])[['Overallscore']].agg('mean')

You can also request more aggregative functions:

In [ ]:
allDataFull.groupby(["Region","Regimetype"])[['Overallscore']].agg(['min','max']).reset_index()

And of course, more variables to aggregate:

In [ ]:
severalVars=['Electoralprocessandpluralism','Functioningofgovernment', 'Politicalparticipation']
allDataFull.groupby(["Region","Regimetype"])[severalVars].agg(['min','max']).reset_index()

The previous column names look more complex:

In [ ]:
multi=allDataFull.groupby(["Region","Regimetype"])[severalVars].agg(['min','max']).reset_index()
multi.columns

The aggregation process can produce **multiIndex**. Most of the time you want to flatten the multiIndex. We could use some basic string operations for that:

In [ ]:
# concatenating:
[(col[0]+'_'+col[1]) for col in multi.columns ]

In [ ]:
# concatenating if secind element is not '':
[(col[0]+'_'+col[1]) if col[1]!='' else col[0] for col in multi.columns ]

In [ ]:
# same as before, but using 'join' function: 
['_'.join(col).strip() if col[1]!='' else col[0] for col in multi.columns ]

I will save the new column names:

In [ ]:
multi.columns= ['_'.join(col).strip() if col[1]!='' else col[0] for col in multi.columns ]

This is the current look:

In [ ]:
multi

The current result can also be used when melting to a long format:

In [ ]:
multi.melt(id_vars=['Region','Regimetype'])

____


# <font color="red">SAVING FILE</font>

The current _allDataFull_ data frame is clean and formatted. We have used CSV files so far:

In [ ]:
allDataFull.to_csv("allDataFull_OK.csv",index=False)

When we read back the file you may notice something wrong:

In [ ]:
allDataFull_OK=pd.read_csv("allDataFull_OK.csv")
allDataFull_OK.info()

We need a file that avoids we have to reformat the columns again and again. Let's see the choices:

#### For future use in Python:

In [ ]:
allDataFull.to_pickle("allDataFull_OK.pkl")
# you will need: DF=pd.read_pickle("allDataFull.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://..../allDataFull.pkl"),compression=None)

In [ ]:
allDataFull_OK=pd.read_pickle("allDataFull_OK.pkl")
allDataFull_OK.info()

In [ ]:
allDataFull_OK.Regimetype

#### For future  use in R:

In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(allDataFull,file="allDataFull_OK.RDS")

#In R, you call it with: DF = readRDS("allDataFull.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://..../allDataFull.RDS")